# Phase 2A: 원본 소스 자동 생성 노트북 (LLM 지식 기반)

이 노트북은 **매장명만 입력**하면 **LLM의 내장 지식**을 활용하여 5가지 소스를 자동 생성합니다.

## LLM 지식 기반 생성

LLM이 학습한 데이터를 바탕으로 다음 5가지 소스를 생성합니다:

1. 🌟 **미쉐린 가이드** - LLM 지식 (별점, 평가)
2. 🔵 **블루리본 서베이** - LLM 지식 (평가, 특징)
3. 👨‍🍳 **셰프 인터뷰** - LLM 지식 (철학, 조리 방식)
4. 🍽️ **코스 구성** - LLM 지식 (메뉴, 시그니처)
5. 🏛️ **철학/공간/전통** - LLM 지식 (브랜드, 인테리어)

## 장점

### 💰 완전 무료
- 웹 크롤링 불필요
- API 호출 비용 없음
- LLM 토큰 비용만 (~$0.02/매장)

### ⚡ 매우 빠름
- 웹 요청 없음
- LLM 응답만 대기
- 총 처리 시간: 10-15초/매장

### 🔧 유지보수 불필요
- 사이트 구조 변경 영향 없음
- 크롤러 관리 불필요
- 안정적 작동

### 📦 간단한 구현
- 복잡한 크롤링 로직 없음
- Selenium/ChromeDriver 불필요
- 코드 100줄 이하

## 주의사항

### ⚠️ 할루시네이션 가능
- LLM이 "학습한 지식"으로 답변
- 실제와 다를 수 있음
- 검증 필요

### ⚠️ 최신 정보 부족
- 2024년 이전 정보 기반
- 신규 오픈 매장 정보 없을 수 있음
- 최신 별점/리뷰 반영 안됨

### ⚠️ 참고용으로 활용
- 초안/프로토타입용
- 중요 정보는 수동 검증 권장
- 공식 인증 정보는 별도 확인

## 적합한 사용 케이스

✅ **프로토타입/POC**
- 빠른 데이터 생성
- 시스템 테스트

✅ **대략적 정보 수집**
- 매장 분류
- 초기 데이터셋 구축

✅ **참고 자료 생성**
- 사람이 보정할 초안
- 일반적인 설명 생성

## 비용

| 항목 | 비용 |
|------|------|
| LLM 토큰 (5개 소스) | ~$0.02/매장 |
| **총 비용** | **$0.02/매장** |

**100개 매장: $2**

## 섹션 1: 패키지 설치

In [ ]:
%pip install google-cloud-aiplatform google-genai python-dotenv pandas

## 섹션 2: 라이브러리 Import

In [ ]:
import os
import json
import time
import vertexai
from google import genai
from google.genai import types
from dotenv import load_dotenv
from datetime import datetime

print("✅ 라이브러리 Import 완료")

## 섹션 3: Vertex AI 초기화

In [ ]:
load_dotenv()

PROJECT_ID = "wad-dw"
LOCATION = "us-central1"

vertexai.init(project=PROJECT_ID, location=LOCATION)
client = genai.Client(vertexai=True, project=PROJECT_ID, location=LOCATION)

print(f"✅ Vertex AI 초기화 완료: {PROJECT_ID} ({LOCATION})")

## 섹션 4: LLM 지식 기반 수집 함수

In [ ]:
# ========================================
# LLM 지식 기반 수집
# ========================================

MODEL_NAME = "gemini-2.5-pro"

print("✅ LLM 지식 기반 모드 초기화 완료")
print(f"   - 모델: {MODEL_NAME}")
print(f"   - 수집 방법: LLM 내장 지식만 사용")
print(f"   - 비용: 토큰 비용만 (~$0.02/매장)")


# ========================================
# 5가지 소스 타입 정의
# ========================================

SOURCE_CONFIGS = {
    "michelin_review": {
        "name": "미쉐린 가이드",
        "prompt": """이 레스토랑에 대한 미쉐린 가이드 정보를 요약하세요:
        - 미쉐린 별점 (0-3스타, 빕 구르망 등)
        - 가격대
        - 주요 평가 내용
        - 레스토랑 특징

        300-800자로 작성하세요. 정보가 확실하지 않으면 found를 false로 설정하세요."""
    },
    "blueribbon_review": {
        "name": "블루리본 서베이",
        "prompt": """이 레스토랑에 대한 블루리본 서베이 정보를 요약하세요:
        - 블루리본 등급/점수
        - 평가 내용
        - 레스토랑 특징

        300-800자로 작성하세요. 정보가 확실하지 않으면 found를 false로 설정하세요."""
    },
    "chef_interview": {
        "name": "셰프 인터뷰",
        "prompt": """이 레스토랑 셰프에 대한 정보를 요약하세요:
        - 셰프 이름 및 경력
        - 요리 철학
        - 식재료 활용 방식
        - 조리 기법 및 스타일

        300-800자로 작성하세요. 정보가 확실하지 않으면 found를 false로 설정하세요."""
    },
    "course_description": {
        "name": "코스 구성",
        "prompt": """이 레스토랑의 메뉴 정보를 요약하세요:
        - 코스 구성 (가능하면 구체적으로)
        - 시그니처 메뉴
        - 대표 요리
        - 가격대

        300-800자로 작성하세요. 정보가 확실하지 않으면 found를 false로 설정하세요."""
    },
    "brand_philosophy": {
        "name": "철학/공간/전통",
        "prompt": """이 레스토랑의 브랜드 정보를 요약하세요:
        - 브랜드 철학 및 스토리
        - 공간 디자인 및 분위기
        - 인테리어 특징
        - 전통 기법 활용

        300-800자로 작성하세요. 정보가 확실하지 않으면 found를 false로 설정하세요."""
    }
}


# ========================================
# LLM 지식 기반 수집 함수
# ========================================

def collect_with_llm_knowledge(shop_name: str, source_type: str, config: dict) -> dict:
    """LLM의 내장 지식을 사용하여 정보 수집"""
    try:
        full_prompt = f"""
        매장명: {shop_name}

        당신의 학습된 지식을 바탕으로 다음 작업을 수행하세요:

        {config['prompt']}

        중요:
        - 학습된 정보만 사용하세요
        - 확실한 내용만 포함하세요
        - 추측하지 마세요
        - 정보가 없거나 확실하지 않으면 found를 false로 설정하세요

        출력 형식 (JSON):
        {{
        "text": "내용...",
        "found": true
        }}
        """
        
        response = client.models.generate_content(
            model=MODEL_NAME,
            contents=full_prompt,
            config=types.GenerateContentConfig(
                temperature=0.3,
                max_output_tokens=2048,
                response_mime_type="application/json"
            )
        )
        
        result = json.loads(response.text)
        
        if result.get("found") and result.get("text"):
            return {
                "success": True,
                "text": result["text"],
                "url": "",
                "method": "llm_knowledge"
            }
        else:
            return {
                "success": False,
                "reason": "LLM이 확실한 정보를 제공할 수 없음"
            }
            
    except Exception as e:
        return {
            "success": False,
            "reason": f"LLM 오류: {str(e)}"
        }


# ========================================
# 매장별 자동 수집 (5가지 소스)
# ========================================

def auto_collect_shop_sources(shop_seq: int, shop_name: str, category: str) -> dict:
    """한 매장에 대해 5가지 소스를 LLM 지식으로 수집"""
    print(f"\n🔍 {shop_name} 정보 수집 중 (LLM 지식)...")
    print("─" * 80)
    
    sources = []
    
    for source_type, config in SOURCE_CONFIGS.items():
        print(f"   [{config['name']}]...", end=" ")
        
        result = collect_with_llm_knowledge(shop_name, source_type, config)
        
        if result["success"]:
            sources.append({
                "source_type": source_type,
                "text": result["text"],
                "url": result.get("url", ""),
                "collected_date": datetime.now().strftime("%Y-%m-%d"),
                "collection_method": "llm_knowledge"
            })
            print(f"✅ ({len(result['text'])}자)")
        else:
            print(f"⚠️ {result['reason']}")
        
        time.sleep(0.5)
    
    return {
        "shop_seq": shop_seq,
        "shop_name": shop_name,
        "category": category,
        "sources": sources
    }


print("\n✅ LLM 지식 기반 수집 함수 정의 완료")
print(f"   - 수집 소스: {len(SOURCE_CONFIGS)}가지")
print(f"   - 모두 LLM 내장 지식 사용")

## 섹션 5: 매장 리스트 입력

수집할 매장의 shop_seq와 shop_name만 입력하세요.

In [ ]:
# ========================================
# 매장 리스트 입력
# ========================================

# 수집할 매장 리스트 (shop_seq + shop_name + category)
BATCH_SHOPS = [
    {"shop_seq": 1, "shop_name": "모수", "category": "fine_dining"},
    {"shop_seq": 2, "shop_name": "밍글스", "category": "fine_dining"},
    {"shop_seq": 3, "shop_name": "스시 소라", "category": "fine_dining"},
    # 추가 매장을 여기에 입력하세요...
]

print(f"📋 자동 수집 대상 매장")
print("=" * 80)
print(f"\n총 {len(BATCH_SHOPS)}개 매장:\n")

for idx, shop in enumerate(BATCH_SHOPS, 1):
    print(f"   [{idx}] {shop['shop_name']} (seq: {shop['shop_seq']}, category: {shop['category']})")

print(f"\n각 매장당 5가지 소스 자동 수집 (LLM 지식):")
for source_type, config in SOURCE_CONFIGS.items():
    print(f"   • {config['name']}")

print("\n" + "=" * 80)

## 섹션 6: LLM 지식 기반 자동 수집 실행

⚠️ LLM의 내장 지식만 사용하여 5가지 소스를 자동 생성합니다.

In [ ]:
# ========================================
# LLM 지식 기반 자동 수집 실행
# ========================================

print("🚀 LLM 지식 기반 자동 수집 시작\n")
print("=" * 80)
print("📚 모든 소스: LLM 내장 지식 활용")
print("⚡ 빠른 처리: 10-15초/매장")
print("💰 저렴한 비용: ~$0.02/매장")
print("=" * 80)

collected_data = []
collection_stats = []

start_time = time.time()

for idx, shop in enumerate(BATCH_SHOPS, 1):
    print(f"\n[{idx}/{len(BATCH_SHOPS)}] {shop['shop_name']} ({shop['category']})")
    print("=" * 80)
    
    shop_start = time.time()
    
    try:
        # LLM 지식 기반 수집 실행
        result = auto_collect_shop_sources(
            shop_seq=shop["shop_seq"],
            shop_name=shop["shop_name"],
            category=shop["category"]
        )
        
        shop_elapsed = time.time() - shop_start
        
        # 통계
        num_sources = len(result.get("sources", []))
        total_chars = sum(len(s["text"]) for s in result.get("sources", []))
        
        print(f"\n✅ 완료: {num_sources}/5개 소스 수집 ({shop_elapsed:.1f}초)")
        print(f"   - 카테고리: {result.get('category')}")
        print(f"   - 총 텍스트: {total_chars:,}자")
        
        collected_data.append(result)
        collection_stats.append({
            "shop_name": shop["shop_name"],
            "category": shop["category"],
            "status": "success",
            "num_sources": num_sources,
            "total_chars": total_chars,
            "elapsed_time": shop_elapsed
        })
        
    except Exception as e:
        print(f"\n❌ 오류: {e}")
        collection_stats.append({
            "shop_name": shop["shop_name"],
            "category": shop.get("category", "unknown"),
            "status": "error",
            "error": str(e)
        })
    
    # 다음 매장까지 대기
    if idx < len(BATCH_SHOPS):
        wait_seconds = 2
        print(f"\n⏳ 다음 매장까지 {wait_seconds}초 대기...")
        time.sleep(wait_seconds)

total_elapsed = time.time() - start_time

print("\n" + "=" * 80)
print(f"\n✅ LLM 지식 기반 수집 완료 (총 {total_elapsed:.1f}초 = {total_elapsed/60:.1f}분)\n")

# 통계
success_count = sum(1 for s in collection_stats if s.get("status") == "success")
total_sources = sum(s.get("num_sources", 0) for s in collection_stats)

print(f"📊 통계:")
print(f"   - 성공: {success_count}/{len(BATCH_SHOPS)}개 매장")
print(f"   - 총 소스: {total_sources}개 (평균 {total_sources/success_count if success_count > 0 else 0:.1f}개/매장)")
print(f"   - 평균 처리 시간: {total_elapsed / len(BATCH_SHOPS):.1f}초/매장")

# 카테고리별 분포
if collected_data:
    category_counts = {}
    for data in collected_data:
        cat = data.get("category", "unknown")
        category_counts[cat] = category_counts.get(cat, 0) + 1
    
    print(f"\n   카테고리별:")
    for cat, count in category_counts.items():
        print(f"   - {cat}: {count}개")

# 비용 계산
llm_tokens_estimate = total_sources * 2000  # 소스당 약 2K tokens
cost_estimate = (llm_tokens_estimate / 1000000) * 1.25  # Input token cost for Gemini 2.5 Pro
print(f"\n💰 예상 비용:")
print(f"   - LLM 토큰: 약 ${cost_estimate:.3f}")
print(f"   - **총 비용: 약 ${cost_estimate:.3f}** (매장당 ${cost_estimate/success_count if success_count > 0 else 0:.3f})")

print("\n" + "=" * 80)

## 다음 단계

### 워크플로우
1. ✅ **source_generator.ipynb** (현재) - 매장명 입력 → LLM 지식 생성 → JSON
2. ⏭️ **source_validator.ipynb** - JSON 검증 및 통합 (선택)
3. ⏭️ **source_indexer.ipynb** - 청킹 및 벡터 DB 인덱싱

### 생성된 파일
- `collected_sources_llm_[timestamp].json` - 생성된 원본 소스 데이터
- `collection_stats_[timestamp].csv` - 생성 통계

### LLM 지식 기반 방식 요약

| 소스 | 생성 방법 | 정확도 | 비용 |
|------|----------|--------|------|
| 🌟 미쉐린 가이드 | LLM 지식 | ⭐⭐⭐ | ~$0.004 |
| 🔵 블루리본 서베이 | LLM 지식 | ⭐⭐⭐ | ~$0.004 |
| 👨‍🍳 셰프 인터뷰 | LLM 지식 | ⭐⭐⭐ | ~$0.004 |
| 🍽️ 코스 구성 | LLM 지식 | ⭐⭐⭐ | ~$0.004 |
| 🏛️ 철학/공간/전통 | LLM 지식 | ⭐⭐⭐ | ~$0.004 |

**총 비용: 매장당 약 $0.02 (100개 = $2)**

### 장점
- ✅ **완전 무료 인프라** (API 호출 없음, 크롤링 없음)
- ✅ **빠른 처리** (10-15초/매장)
- ✅ **유지보수 불필요** (사이트 변경 영향 없음)
- ✅ **간단한 코드** (100줄 이하)

### 주의사항
- ⚠️ **할루시네이션 가능** - LLM이 잘못된 정보 생성할 수 있음
- ⚠️ **최신 정보 부족** - 2024년 이전 데이터 기반
- ⚠️ **참고용으로 활용** - 중요 정보는 수동 검증 필요

### 적합한 사용 케이스
- 프로토타입/POC
- 대략적 정보 수집
- 사람이 보정할 초안 생성

In [ ]:
# ========================================
# 수집 품질 검증
# ========================================

if collected_data:
    print("🔍 수집 품질 검증\n")
    print("=" * 80)
    
    for shop in collected_data:
        shop_name = shop.get("shop_name", "Unknown")
        category = shop.get("category", "unknown")
        sources = shop.get("sources", [])
        
        print(f"\n📍 {shop_name} ({category})")
        print("─" * 80)
        
        # 소스 타입별 통계
        source_types = {}
        total_chars = 0
        
        for source in sources:
            stype = source.get("source_type", "unknown")
            text = source.get("text", "")
            text_len = len(text)
            
            source_types[stype] = source_types.get(stype, 0) + 1
            total_chars += text_len
        
        print(f"   총 소스: {len(sources)}/5개")
        print(f"   총 텍스트: {total_chars:,}자")
        print(f"   평균 길이: {total_chars // len(sources) if sources else 0:,}자/소스")
        
        print(f"\n   수집된 소스:")
        for stype, count in sorted(source_types.items()):
            source_name = SOURCE_CONFIGS.get(stype, {}).get("name", stype)
            print(f"      ✅ {source_name}: {count}개")
        
        # 누락된 소스 확인
        missing = set(SOURCE_CONFIGS.keys()) - set(source_types.keys())
        if missing:
            print(f"\n   ⚠️ 누락된 소스:")
            for stype in missing:
                source_name = SOURCE_CONFIGS[stype]["name"]
                print(f"      ❌ {source_name}")
        
        # 경고
        if len(sources) < 3:
            print(f"\n   ⚠️ 소스가 너무 적습니다 (최소 3개 권장)")
        
        if total_chars < 500:
            print(f"\n   ⚠️ 전체 텍스트가 너무 짧습니다 (최소 500자 권장)")
    
    print("\n" + "=" * 80)
else:
    print("⚠️ 검증할 데이터가 없습니다.")

## 섹션 8: 수집 품질 검증 (선택)

In [ ]:
# ========================================
# JSON 파일 저장
# ========================================

if collected_data:
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"collected_sources_llm_{timestamp}.json"
    
    # JSON 저장
    with open(filename, 'w', encoding='utf-8') as f:
        json.dump(collected_data, f, ensure_ascii=False, indent=2)
    
    file_size = len(json.dumps(collected_data, ensure_ascii=False))
    
    print(f"💾 JSON 저장 완료: {filename}")
    print(f"   - 매장 수: {len(collected_data)}개")
    print(f"   - 파일 크기: {file_size:,} bytes ({file_size / 1024:.1f} KB)")
    
    # 통계 CSV도 저장
    import pandas as pd
    df_stats = pd.DataFrame(collection_stats)
    csv_filename = f"collection_stats_{timestamp}.csv"
    df_stats.to_csv(csv_filename, index=False, encoding='utf-8-sig')
    
    print(f"📊 통계 파일 저장: {csv_filename}")
    
    print(f"\n✅ 다음 단계:")
    print(f"   1. source_validator.ipynb에서 검증 (선택)")
    print(f"   2. source_indexer.ipynb에서 벡터 DB 인덱싱")
else:
    print("⚠️ 저장할 데이터가 없습니다.")

## 섹션 7: JSON 파일로 저장